In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count,col, when,countDistinct,mean

spark = SparkSession.builder.getOrCreate()

In [13]:
listings_df = spark.read.option("header", "true").option("sep", ",").csv("./work/data/listings.csv")
loyer_df = spark.read.option("header", "true").option("sep",";").csv("./work/data/loyers.csv")

In [ ]:
# count property_type
listings_df.groupBy("property_type").agg(count("property_type").alias("count")).sort("count", ascending=False).show(5)

+--------------------+-----+
|       property_type|count|
+--------------------+-----+
|  Entire rental unit|58124|
|Private room in r...| 4582|
|        Entire condo| 1921|
|         ""Kitchen""| 1643|
|      ""Bed linens""| 1100|
+--------------------+-----+
only showing top 5 rows



In [24]:
listings_df.groupBy("room_type").agg(count("room_type").alias("count")).sort("count", ascending=False).show(5)

+---------------+-----+
|      room_type|count|
+---------------+-----+
|Entire home/apt|61737|
|   Private room| 6692|
|    ""Kitchen""| 1553|
| ""Bed linens""| 1072|
|       ""Wifi""| 1002|
+---------------+-----+
only showing top 5 rows



In [26]:
listings_filter_cols_df = listings_df.select("id","room_type","price","availability_365", "longitude", "latitude", "property_type", "room_type", "accommodates", "bathrooms", "bedrooms", "beds", "amenities", "availability_30", "availability_60", "availability_90", "availability_365")
listings_filter_cols_df = listings_filter_cols_df.filter(col("property_type") == "Entire rental unit").filter(col("room_type") == "Entire home/apt")
# filter na in cols 

In [32]:
# parse bathrooms, bedrooms, beds to double or na
listings_filter_cols_df = listings_filter_cols_df.withColumn("bathrooms", col("bathrooms").cast("double"))
listings_filter_cols_df = listings_filter_cols_df.withColumn("bedrooms", col("bedrooms").cast("double"))
listings_filter_cols_df = listings_filter_cols_df.withColumn("beds", col("beds").cast("double"))
# filter na in bathroom, bedrooms, beds
listings_filter_cols_df = listings_filter_cols_df.filter(col("bathrooms").isNotNull())
listings_filter_cols_df = listings_filter_cols_df.filter(col("bedrooms").isNotNull())
listings_filter_cols_df = listings_filter_cols_df.filter(col("beds").isNotNull())

# number of rows
print("Number of rows: ", listings_filter_cols_df.count())

Number of rows:  39429


In [36]:
# create number of rooms column
listings_filter_cols_df = listings_filter_cols_df.withColumn("number_of_rooms", col("bedrooms") + col("bathrooms") + 1) # add 1 for living room
# print mean number of rooms and min and max
listings_filter_cols_df.agg(mean("number_of_rooms").alias("mean_number_of_rooms")).show()
listings_filter_cols_df.agg({"number_of_rooms": "min"}).show()
listings_filter_cols_df.agg({"number_of_rooms": "max"}).show()

+--------------------+
|mean_number_of_rooms|
+--------------------+
|  3.4614370133657966|
+--------------------+

+--------------------+
|min(number_of_rooms)|
+--------------------+
|                 1.0|
+--------------------+

+--------------------+
|max(number_of_rooms)|
+--------------------+
|                34.0|
+--------------------+



In [6]:
loyer_filter_cols_df = loyer_df.select("ref", "piece", "meuble_txt", "max", "min", "ville", "geo_point_2d")

root
 |-- id: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- availability_365: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- beds: string (nullable = true)
 |-- amenities: string (nullable = true)
 |-- availability_30: string (nullable = true)
 |-- availability_60: string (nullable = true)
 |-- availability_90: string (nullable = true)
 |-- availability_365: string (nullable = true)



+--------------------+
|       property_type|
+--------------------+
|      ""Essentials""|
|                 1.0|
|     ""Smoke alarm""|
|   ""Portable fans""|
|Private room in loft|
|            48.83101|
|             2.34428|
|https://www.airbn...|
|         ""Heating""|
|                   7|
| ""HDTV with stan...|
|  Amazon Prime Video|
|Private room in e...|
|             2.32929|
| ""Clothing stora...|
|https://a0.muscac...|
|https://a0.muscac...|
|             2.34318|
|             Reuilly|
|            48.84696|
+--------------------+
only showing top 20 rows

